## Example Workflow

Using the `'json'` dumpled from a `'Workflow'` dataclass create a DAG.

In [36]:
import ipycytoscape
from dataclasses import dataclass, asdict

In [37]:
# Dataclasses
from typing import Optional

from workflow.tasks import Task


@dataclass
class Step:
    step_id: str
    task_name: str
    target: Optional[str]
    start: Optional[bool] = None
    task_info: Optional[dict] = None

    @property
    def task(self):
        return Task.tasks[self.task_name]


@dataclass
class Workflow:
    name: str
    steps: list[Step]

    def get_step(self, step_id):
        return next(step for step in self.steps if step.step_id == step_id)

    @property
    def first_step(self):
        return next(step for step in self.steps if step.start)

In [38]:
steps=[
        Step(
            step_id="approver_email_form",
            task_name="email_form",
            start=True,
            target=["send_approver_email", "send_approver_sms"],
            task_info={"subject": "Approval"},
        ),
    Step(
            step_id="send_approver_sms",
            task_name="send_email",
            target="approver_email_form2",
            task_info={
                "subject": "Test subject 3",
                "message": "Test message 3",
                "from_email": "admin3@example.com",
                "recipient_list": ["approver3@example.com"],
            },
        ),
   
    Step(
            step_id="send_approver_email",
            task_name="send_email",
            target="approver_email_form2",
            task_info={
                "subject": "Test subject 3",
                "message": "Test message 3",
                "from_email": "admin3@example.com",
                "recipient_list": ["approver3@example.com"],
            },
        ),
        Step(
            step_id="approver_email_form2",
            task_name="email_form",
            target="send_approver_email2",
            task_info={"subject": "Approval"},
        ),
        Step(
            step_id="send_approver_email2",
            task_name="send_email",
            target=None,
            task_info={
                "subject": "Test subject 4",
                "message": "Test message 4",
                "from_email": "admin3@example.com",
                "recipient_list": ["approver3@example.com"],
            },
        ),
    ]

In [39]:
# An example workflow
ExampleWorkflow = Workflow(name="example_workflow", steps=steps)

In [40]:
# Nodes
def step_to_node(step):
    return {
        "data": {
            "id": step.step_id,
            "label": step.task_name,
            "classes": 'top-left',
        }
    }

nodes = [step_to_node(step) for step in steps]

In [41]:
# Edges
from itertools import permutations

edges = []
for step in steps:
    targets = step.target if isinstance(step.target, list) else [step.target]
    for target in targets:
        edges.append(
            {
                'data': {
                    'source': step.step_id,
                    'target': target
                }
            }
        )

In [42]:
data = {
    'nodes': nodes,
    'edges': edges
}

In [43]:
cytoscapeobj = ipycytoscape.CytoscapeWidget()
cytoscapeobj.graph.add_graph_from_json(data)

In [44]:
cytoscapeobj.set_style([
    {
        'selector': 'node',
        'css': {
            'background-color': '#11479e'
        }
    },
    {
        'selector': 'node:parent',
        'css': {
            'background-opacity': 0.333
        }
    },
    {
        'selector': 'edge',
        'style': {
            'width': 4,
            'line-color': 'blue',
            'target-arrow-shape': 'triangle',
            'target-arrow-color': '#9dbaea',
            'curve-style': 'bezier'
        }
    },
    {
        'selector': 'node',
        'css': {
            'background-color': 'lightblue',
            'shape': 'roundrectangle',
            'width': '100',
            'height': '20',
            'border-color': 'blue',
        }
    },
    {
        'selector': 'node[classes]',
        'style': {
            'font-family': 'helvetica',
            'font-size': '20px',
            'color': 'blue',
            'text-valign': 'top',
            'text-halign': 'left'
        }
    },
   
    {
        'selector': 'node[classes]',
        'style': {
            'label': 'data(label)'
        }
    },
    {
        'selector': 'edge',
        'css': {
            'line-color': 'black',
            'width': 2,
            
        }
    },
])

In [45]:
cytoscapeobj

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'css': {'background-c…